In [2]:
%cd /content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker

/content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker


In [20]:
import pickle
import pandas as pd
import numpy as np
import lightgbm as lgb

In [22]:
train_data_path = '../../data/train.pkl'
valid_data_path = '../../data/valid.pkl'
data_file_format = "pickle"
feature_cols = ['price', 'sales_channel_id']
sample_weight_col = 'buy'
target_col = 'buy'
group_record_cols = ['customer_id']
eval_metric='ndcg'
eval_at=2
model_output_path='./model.pkl'

In [7]:
if data_file_format == "csv":
  train = pd.read_csv(train_data_path)
  valid = pd.read_csv(valid_data_path)  
elif data_file_format == "parquet":
  train = pd.read_parquet(train_data_path)
  valid = pd.read_parquet(valid_data_path)  
elif data_file_format == "pickle":
  train = pd.read_pickle(train_data_path)
  valid = pd.read_pickle(valid_data_path)
else:
  train = pd.read_csv(train_data_path)
  valid = pd.read_csv(valid_data_path)

train = train.sort_values(group_record_cols)
valid = valid.sort_values(group_record_cols)
train_group=train.groupby(group_record_cols)[group_record_cols[0]].count().to_numpy()
valid_group=valid.groupby(group_record_cols)[group_record_cols[0]].count().to_numpy()

In [12]:
model = lgb.LGBMRanker(
    objective="lambdarank",
    boosting_type='gbdt', 
    metric=eval_metric,
    num_leaves=2**7-1,
    max_depth=7, 
    learning_rate=0.1, 
    n_estimators=100,
    early_stopping_rounds=10,
    random_state=42,
)

In [ ]:
model.fit(
    X=train[feature_cols],
    y=train[target_col],
    sample_weight=train[sample_weight_col],
    group=train_group,
    eval_set=[(valid[feature_cols], valid[target_col])],
    eval_names=['valid'],
    eval_sample_weight=[valid[sample_weight_col]],
    eval_at=eval_at,
    eval_group=[valid_group],
    )

In [25]:
with open(model_output_path, 'wb') as f:
  pickle.dump(model, f)